In [17]:
minimum_precision = 4
# THIS IS AN INCLUSIVE RANGE!!
maximum_precision = 17

In [18]:
header = """
use hyperloglog_rs::prelude::*;

"""

test = """
#[test]
pub fn test_hyper_log_log_at_precision_{precision}_and_bits_{bits}() {{
    type PRECISION = Precision{precision};
    const BITS: usize = {bits};
    
    for number_of_elements in [
        5, 10, 15, 100, 200, 1000, 10_000, 100_000, 1_000_000
    ]{{
        if BITS <= 4 && {precision} <= 5 && number_of_elements > 10_000{{
            continue;
        }}
    
        let mut hll: HyperLogLog<PRECISION, BITS, SipHasher13> = HyperLogLog::new();
        let hll_default: HyperLogLog<PRECISION, BITS, SipHasher13> = HyperLogLog::default();
        
        assert_eq!(hll, hll_default);
        
        assert_eq!(hll.get_number_of_bits(), BITS);
        assert!(hll.is_empty());

        for i in 0..number_of_elements {{
            hll.insert(i);
            assert!(hll.may_contain(&i));
        }}
        
        assert!(!hll.is_empty());

        assert!(
            hll.estimate_cardinality() >= number_of_elements as f32 * 7.0_f32 / 10.0_f32,
            concat!(
                "Obtained: {{}}, Expected around: {{}}. ",
            ),
            hll.estimate_cardinality(), number_of_elements,
        );

        assert!(
            hll.estimate_cardinality() <= number_of_elements as f32 * 14.0_f32 / 10.0_f32,
            concat!(
                "Obtained: {{}}, Expected around: {{}}. ",
            ),
            hll.estimate_cardinality(), number_of_elements,
        );
    }}
}}
"""

In [19]:
for precision in range(minimum_precision, maximum_precision + 1):
    tests=[]
    for bits_per_register in (4, 5, 6):
        tests.append(test.format(
            precision=precision,
            bits=bits_per_register
        ))
    with open(f"tests/test_hll_{precision}.rs", "w") as f:
        f.write(header + "\n\n".join(tests))